In [14]:
# import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import v2
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import glob
import tqdm


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
torch.cuda.is_available()

False

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# !unzip gdrive/My\ Drive/project_data.zip -d ./data

In [16]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        self.filepaths = glob.glob(os.path.join(data_dir, '**/*.jpg'), recursive=True)
        self.classes = ['real', 'fake']

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, index):
        filepath = self.filepaths[index]

        # Extract the class name from the filepath
        class_name = self.classes[1] if "Fake" in os.path.dirname(os.path.relpath(filepath, data_dir)) else \
        self.classes[0]

        # Map the class name to 'real' or 'fake'
        class_label = self.classes.index(class_name)

        # Load and transform the image
        image = Image.open(filepath)

        if self.transform is not None:
            image = self.transform(image)

        return image, class_label

In [17]:
def test(model_ft, data_path):
  # Data transform

  # Data augmentation and normalization for training
  # Just normalization for validation
  data_transforms = {
      'test': v2.Compose([
          v2.Resize(256),
          v2.CenterCrop(224),
          v2.ToTensor(),
          v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ]),
  }
  all_data = datasets.ImageFolder(root=data_path,
                                    transform=data_transforms['test'],
                                    target_transform=None)

  test_dataloader = DataLoader(dataset=all_data,
                            batch_size=10,
                            num_workers=1,
                            shuffle=False)
  model_ft.to(device)
  model_ft.eval()
  list_preds = None
  list_labels = None


  with torch.no_grad():
    for inputs, labels in tqdm(test_dataloader):
      labels = labels.to(device, dtype=torch.float32)
      inputs = inputs.to(device, dtype=torch.float32)

      outputs = model_ft(inputs)
      _, preds = torch.max(outputs, 1)

      if list_preds is not None:
        list_preds = torch.hstack((list_preds, preds.cpu()))
        list_labels = torch.hstack((list_labels, labels.cpu()))
      else:
        list_preds = preds.cpu()
        list_labels = labels.cpu()


  # after you calculate AUC
  list_preds = list_preds > 0.5
  list_preds = list_preds.long()
  list_labels = list_labels.long()

  acc = torch.sum(list_preds == list_labels) / len(all_data)


  return acc.item()





In [19]:
PATH_TO_DATA = './data'
PATH_TO_MODEL = './resnet50.pt'

In [20]:
model = torch.load(PATH_TO_MODEL)

acc = test(model, PATH_TO_DATA)

print(f'Accuracy : {acc}')

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


AttributeError: ignored